In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\Admin\\anaconda3\\envs\\SparkEnvironment\\Lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
    .builder
    .appName("UDFApp")
    .master("local[4]")
    .config("spark.dynamicAloocation.enabled", "false")
    .getOrCreate()
)

sc = spark.sparkContext

spark

In [7]:
cabsDf = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("C:\DataFiles\Cabs.csv")
)
cabsDf.createOrReplaceTempView("Cabs")
cabsDf.show(truncate = False)

+---------+--------------------+------------------------+----------------+------+-------------------+-----------------+--------------------+-----------+-----------+---------------+--------------------------+--------------------------------------------+---------------+
|CabNumber|VehicleLicenseNumber|Name                    |LicenseType     |Active|PermitLicenseNumber|VehicleVinNumber |WheelchairAccessible|VehicleYear|VehicleType|TelephoneNumber|Website                   |Address                                     |LastDateUpdated|
+---------+--------------------+------------------------+----------------+------+-------------------+-----------------+--------------------+-----------+-----------+---------------+--------------------------+--------------------------------------------+---------------+
|T802127C |C19641              |ABCON INC.              |OWNER MUST DRIVE|YES   |NULL               |5TDBK3EH0DS268018|NULL                |2016       |NULL       |(718)438-1100  |NULL         

In [14]:
def convertCase(str):
    result = ""
    nameWordsArray = str.split(",")
    for nameWord in nameWordsArray:
        result = (result
                  + nameWord[0:1].upper()
                  + nameWord[1:len(nameWord)].lower()
                  +","
                 )
        result = result[0:len(result) -2]
        return result

In [15]:
convertCaseUdf = udf(lambda str: convertCase(str), StringType())

In [17]:
(
    cabsDf
    .select(
        "Name",
        convertCaseUdf(col("Name")).alias("Name_convertCase")
    )
).show(truncate = False)

+------------------------+-----------------------+
|Name                    |Name_convertCase       |
+------------------------+-----------------------+
|ABCON INC.              |Abcon inc              |
|ACCEPTABLE TAXI LLC     |Acceptable taxi ll     |
|ALLIS CAB CORP          |Allis cab cor          |
|BENE CAB CORP           |Bene cab cor           |
|BOULOS TAXI CORP.       |Boulos taxi corp       |
|CACERES,JAIME,P         |Cacere                 |
|CALCIUM ONE SERVICE INC.|Calcium one service inc|
|CHARLES,WILBERT         |Charle                 |
|CHAWKI,MICHAEL          |Chawk                  |
|CHRYSOVALANTOU CORP,    |Chrysovalantou cor     |
|COFI BOAT CORP.         |Cofi boat corp         |
|DEKEL TAXI CAB CORP     |Dekel taxi cab cor     |
|FLORIAN & ROBERT INC    |Florian & robert in    |
|GART CAB CORP           |Gart cab cor           |
|GAUTHIER,JACQUES        |Gauthie                |
|GEORGAKOPOULOS, GEORGIA |Georgakopoulo          |
|GUJAR CAB CORP          |Gujar